In [1]:
# loading essential libraries
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV,cross_val_score
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense,Flatten,Dropout,Conv2D,MaxPooling2D
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [2]:
training_data = pd.read_csv('Datasets/Train.csv')
training_data
testing_data = pd.read_csv('Datasets/Test.csv')
testing_data

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [3]:
X_train = training_data.loc[:,[ 'Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']]
Y_train = training_data['Survived']
X_test = testing_data.loc[:,[ 'Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']]

In [4]:
X_test['Embarked'].mode()

0    S
dtype: object

In [5]:
X_train['Sex'] = X_train['Sex'].replace({
    'female' : 1,
    'male' : 0
})
X_test['Sex'] = X_test['Sex'].replace({
    'female' : 1,
    'male' : 0
})

In [6]:
X_train['Age'].fillna(X_train['Age'].mean(),inplace=True)
X_test['Age'].fillna(X_train['Age'].mean(),inplace=True)

In [7]:
X_train['Age'].isnull().sum()
X_test['Age'].isnull().sum()

0

In [8]:
X_train['Embarked'].fillna('S',inplace=True)
X_test['Embarked'].fillna('S',inplace=True)

In [9]:
X_train['Embarked'].isnull().sum()
X_test['Embarked'].isnull().sum()

0

In [10]:
X_train['is_S_Embarked'] = X_train['Embarked'].apply(lambda x : 'S' in x )
X_train['is_Q_Embarked'] = X_train['Embarked'].apply(lambda x : 'Q' in x )
X_train['is_C_Embarked'] = X_train['Embarked'].apply(lambda x : 'C' in x )

X_test['is_S_Embarked'] = X_test['Embarked'].apply(lambda x : 'S' in x )
X_test['is_Q_Embarked'] = X_test['Embarked'].apply(lambda x : 'Q' in x )
X_test['is_C_Embarked'] = X_test['Embarked'].apply(lambda x : 'C' in x )

In [11]:
del X_train['Embarked']
del X_test['Embarked']

In [12]:
X_train['is_S_Embarked'] = X_train['is_S_Embarked'].replace({
    True : 1,
    False:0
})
X_train['is_Q_Embarked'] = X_train['is_Q_Embarked'].replace({
    True : 1,
    False:0
})
X_train['is_C_Embarked'] = X_train['is_C_Embarked'].replace({
    True : 1,
    False:0
})
X_train

,Pclass,Sex,Age,SibSp,Parch,Fare,is_S_Embarked,is_Q_Embarked,is_C_Embarked
0,3,0,22.000000,1,0,7.2500,1,0,0
1,1,1,38.000000,1,0,71.2833,0,0,1
2,3,1,26.000000,0,0,7.9250,1,0,0
3,1,1,35.000000,1,0,53.1000,1,0,0
4,3,0,35.000000,0,0,8.0500,1,0,0
...,...,...,...,...,...,...,...,...,...
886,2,0,27.000000,0,0,13.0000,1,0,0
887,1,1,19.000000,0,0,30.0000,1,0,0
888,3,1,29.699118,1,2,23.4500,1,0,0
889,1,0,26.000000,0,0,30.0000,0,0,1


In [13]:
X_test['is_S_Embarked'] = X_test['is_S_Embarked'].replace({
    True : 1,
    False:0
})
X_test['is_Q_Embarked'] = X_test['is_Q_Embarked'].replace({
    True : 1,
    False:0
})
X_test['is_C_Embarked'] = X_test['is_C_Embarked'].replace({
    True : 1,
    False:0
})
X_test

,Pclass,Sex,Age,SibSp,Parch,Fare,is_S_Embarked,is_Q_Embarked,is_C_Embarked
0,3,0,34.500000,0,0,7.8292,0,1,0
1,3,1,47.000000,1,0,7.0000,1,0,0
2,2,0,62.000000,0,0,9.6875,0,1,0
3,3,0,27.000000,0,0,8.6625,1,0,0
4,3,1,22.000000,1,1,12.2875,1,0,0
...,...,...,...,...,...,...,...,...,...
413,3,0,29.699118,0,0,8.0500,1,0,0
414,1,1,39.000000,0,0,108.9000,0,0,1
415,3,0,38.500000,0,0,7.2500,1,0,0
416,3,0,29.699118,0,0,8.0500,1,0,0


In [14]:
# Best
model2 = RandomForestClassifier(n_estimators=1000)
cross_val_score(model2,X_train,Y_train).mean()

0.813709120582512

In [15]:
model4 = DecisionTreeClassifier()
cross_val_score(model4,X_train,Y_train).mean()

0.7587533739250517

In [16]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [17]:
X_train.shape

(891, 9)

In [18]:
Y_train.shape

(891,)

In [19]:
model1 = LogisticRegression()
cross_val_score(model1,X_train,Y_train).mean()

0.7867679367271359

In [20]:
# model2 = RandomForestClassifier()
# grid = GridSearchCV(model2,{
#     'n_estimators':[100,500,1000,1500,2000]
# })
# grid.fit(X_train,Y_train)
# grid.best_estimator_

In [21]:
# model3 = SVC()
# grid = GridSearchCV(model3,{
#     'C':[1,10,100,1000,10000],
#     'gamma':[0.001,0.005,0.01,0.05]
# })
# grid.fit(X_train,Y_train)
# grid.best_estimator_

In [22]:
model3 =SVC(C=10000, gamma=0.001)
cross_val_score(model3,X_train,Y_train).mean()

0.82712949595129

In [23]:
model5 = KNeighborsClassifier(n_neighbors=9)
cross_val_score(model4,X_train,Y_train).mean()

0.7744146632352018

In [36]:
model6 = Sequential()
model6.add(Dense(12,activation='relu',input_dim=9))
model6.add(Dense(9,activation='relu'))
model6.add(Dense(4,activation='relu'))
model6.add(Dense(2,activation='relu'))
model6.add(Dense(1,activation='sigmoid'))

In [37]:
model6.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'] )

In [38]:
model6.fit(X_train,Y_train,epochs=300,batch_size=30)

Epoch 1/300
30/30 [==============================] - 0s 2ms/step - loss: 0.7007 - accuracy: 0.5511
Epoch 2/300
30/30 [==============================] - 0s 2ms/step - loss: 0.6653 - accuracy: 0.6723
Epoch 3/300
30/30 [==============================] - 0s 1ms/step - loss: 0.6393 - accuracy: 0.7048
Epoch 4/300
30/30 [==============================] - 0s 2ms/step - loss: 0.6065 - accuracy: 0.7306
Epoch 5/300
30/30 [==============================] - 0s 2ms/step - loss: 0.5672 - accuracy: 0.7598
Epoch 6/300
30/30 [==============================] - 0s 2ms/step - loss: 0.5285 - accuracy: 0.7722
Epoch 7/300
30/30 [==============================] - 0s 2ms/step - loss: 0.5001 - accuracy: 0.7722
Epoch 8/300
30/30 [==============================] - 0s 2ms/step - loss: 0.4821 - accuracy: 0.7811
Epoch 9/300
30/30 [==============================] - 0s 2ms/step - loss: 0.4673 - accuracy: 0.7856
Epoch 10/300
30/30 [==============================] - 0s 2ms/step - loss: 0.4581 - accuracy: 0.7868
Epoch 11/

30/30 [==============================] - 0s 3ms/step - loss: 0.3621 - accuracy: 0.8496
Epoch 84/300
30/30 [==============================] - 0s 1ms/step - loss: 0.3618 - accuracy: 0.8530
Epoch 85/300
30/30 [==============================] - 0s 2ms/step - loss: 0.3609 - accuracy: 0.8519
Epoch 86/300
30/30 [==============================] - 0s 1ms/step - loss: 0.3610 - accuracy: 0.8496
Epoch 87/300
30/30 [==============================] - 0s 1ms/step - loss: 0.3599 - accuracy: 0.8519
Epoch 88/300
30/30 [==============================] - 0s 2ms/step - loss: 0.3601 - accuracy: 0.8496
Epoch 89/300
30/30 [==============================] - 0s 1ms/step - loss: 0.3603 - accuracy: 0.8552
Epoch 90/300
30/30 [==============================] - 0s 1ms/step - loss: 0.3591 - accuracy: 0.8541
Epoch 91/300
30/30 [==============================] - 0s 1ms/step - loss: 0.3582 - accuracy: 0.8563
Epoch 92/300
30/30 [==============================] - 0s 2ms/step - loss: 0.3566 - accuracy: 0.8552
Epoch 93/300


30/30 [==============================] - 0s 2ms/step - loss: 0.3384 - accuracy: 0.8631
Epoch 165/300
30/30 [==============================] - 0s 2ms/step - loss: 0.3375 - accuracy: 0.8698
Epoch 166/300
30/30 [==============================] - 0s 2ms/step - loss: 0.3376 - accuracy: 0.8620
Epoch 167/300
30/30 [==============================] - 0s 2ms/step - loss: 0.3367 - accuracy: 0.8642
Epoch 168/300
30/30 [==============================] - 0s 2ms/step - loss: 0.3361 - accuracy: 0.8653
Epoch 169/300
30/30 [==============================] - 0s 2ms/step - loss: 0.3361 - accuracy: 0.8687
Epoch 170/300
30/30 [==============================] - 0s 2ms/step - loss: 0.3362 - accuracy: 0.8642
Epoch 171/300
30/30 [==============================] - 0s 2ms/step - loss: 0.3367 - accuracy: 0.8608
Epoch 172/300
30/30 [==============================] - 0s 2ms/step - loss: 0.3359 - accuracy: 0.8642
Epoch 173/300
30/30 [==============================] - 0s 2ms/step - loss: 0.3350 - accuracy: 0.8653
Epoc

30/30 [==============================] - 0s 2ms/step - loss: 0.3213 - accuracy: 0.8676
Epoch 246/300
30/30 [==============================] - 0s 2ms/step - loss: 0.3240 - accuracy: 0.8687
Epoch 247/300
30/30 [==============================] - 0s 2ms/step - loss: 0.3234 - accuracy: 0.8676
Epoch 248/300
30/30 [==============================] - 0s 2ms/step - loss: 0.3246 - accuracy: 0.8709
Epoch 249/300
30/30 [==============================] - 0s 2ms/step - loss: 0.3239 - accuracy: 0.8664
Epoch 250/300
30/30 [==============================] - 0s 2ms/step - loss: 0.3214 - accuracy: 0.8721
Epoch 251/300
30/30 [==============================] - 0s 2ms/step - loss: 0.3213 - accuracy: 0.8687
Epoch 252/300
30/30 [==============================] - 0s 2ms/step - loss: 0.3201 - accuracy: 0.8698
Epoch 253/300
30/30 [==============================] - 0s 2ms/step - loss: 0.3215 - accuracy: 0.8698
Epoch 254/300
30/30 [==============================] - 0s 1ms/step - loss: 0.3195 - accuracy: 0.8698
Epoc

In [40]:
ans = model6.predict_classes(X_test)
predictions = []
for i in ans:
    predictions.append(i[0])
predictions

[0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,


In [41]:
# np.savetxt('prediction.csv',predictions,fmt="%.0f",delimiter=',')
# Accuracy : 0.87

In [43]:
pd.DataFrame({
    'PassengerId':testing_data['PassengerId'],
    'Survived':predictions
}).to_csv('prediction.csv',index=False)